In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install pydicom

In [ ]:
##### Could clean this up a bit

%matplotlib inline

import os
import time
import copy
import matplotlib.pyplot as plt
import numpy as np
import random
import pandas as pd
import cv2

import torchvision
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import torchvision.transforms as transforms
from torchvision.io import read_image
from torchvision import datasets, models
from torchvision.utils import make_grid
from torchvision.utils import save_image
from torch.utils import data
from torchsummary import summary

from pathlib import Path
from pydicom import dcmread
from tqdm.notebook import tqdm
import skimage.io as sk

from PIL import Image

#torch.manual_seed(0)

print('Using PyTorch version', torch.__version__)

In [ ]:
##### Specify Device

device = torch.device("cpu")

if torch.cuda.is_available():
   print("Training on GPU")
   device = torch.device("cuda:0")


In [ ]:
#Make Labels
df = pd.read_csv("/content/drive/MyDrive/Saliency Map Research 2023/Pneumonia Dataset/Pneumonia_Labels_Final_2.csv")
df2 = df[['Split','patientId','Target']]

df_train = df2.loc[(df2['Split'] == 'train')]
df_train = df_train.drop(df_train.columns[[0]], axis=1)
df_train = df_train.reset_index(drop=True)

df_val = df2.loc[(df2['Split'] == 'val')]
df_val = df_val.drop(df_val.columns[[0]], axis=1)
df_val = df_val.reset_index(drop=True)

df_test = df2.loc[(df2['Split'] == 'test')]
df_test = df_test.drop(df_test.columns[[0]], axis=1)
df_test = df_test.reset_index(drop=True)

#Reduce Size for testing
#train_total = 200
#val_total = 100
#test_total = 0

#df_train = df_train[:train_total]
#df_val = df_val[:val_total]
#df_test = df_test[:test_total]

#Path Split
root = '/content/drive/MyDrive/Saliency Map Research 2023/Pneumonia Dataset/stage_2_train_images_png'

train_labels = df_train[['patientId', 'Target']].to_numpy()
val_labels = df_val[['patientId', 'Target']].to_numpy()
test_labels = df_test[['patientId', 'Target']].to_numpy()

train_paths = [os.path.join(root, image[0]) for image in train_labels]
val_paths = [os.path.join(root, image[0]) for image in val_labels]
test_paths = [os.path.join(root, image[0]) for image in test_labels]

In [ ]:
#Make Labels
df = pd.read_csv("/content/drive/MyDrive/Saliency Map Research 2023/Pneumothorax Dataset/Pneumothorax_Labels_Final.csv")
df = df.rename(columns={"label": "Target", "ImageId": "patientId"})
df2 = df[['Split','patientId','Target']]

df_train = df2.loc[(df2['Split'] == 'train')]
df_train = df_train.drop(df_train.columns[[0]], axis=1)
df_train = df_train.reset_index(drop=True)

df_val = df2.loc[(df2['Split'] == 'val')]
df_val = df_val.drop(df_val.columns[[0]], axis=1)
df_val = df_val.reset_index(drop=True)

df_test = df2.loc[(df2['Split'] == 'test')]
df_test = df_test.drop(df_test.columns[[0]], axis=1)
df_test = df_test.reset_index(drop=True)

#Reduce Size for testing
#train_total = 200
#val_total = 100
#test_total = 0

#df_train = df_train[:train_total]
#df_val = df_val[:val_total]
#df_test = df_test[:test_total]

#Path Split
root = '/content/drive/MyDrive/Saliency Map Research 2023/Pneumothorax Dataset/images_png/PNG/train'

train_labels = df_train[['patientId', 'Target']].to_numpy()
val_labels = df_val[['patientId', 'Target']].to_numpy()
test_labels = df_test[['patientId', 'Target']].to_numpy()

train_paths = [os.path.join(root, image[0]) for image in train_labels]
val_paths = [os.path.join(root, image[0]) for image in val_labels]
test_paths = [os.path.join(root, image[0]) for image in test_labels]

In [ ]:

#print(f'patientId: {train_labels[0][0]}, Target: {train_labels[0][1]}')
print(len(train_paths))
print(len(val_paths))
print(len(test_paths))

In [ ]:
#### Transformations
transform_v3 = torchvision.transforms.Compose([
    torchvision.transforms.Resize(size=(299)),
    torchvision.transforms.RandomHorizontalFlip(),
    torchvision.transforms.ToTensor(),
    #torchvision.transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

transform_dense = torchvision.transforms.Compose([
    torchvision.transforms.Resize(size=(224)),
    torchvision.transforms.RandomHorizontalFlip(),
    torchvision.transforms.ToTensor(),
    #torchvision.transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

###### READ JPG ######

class PneumoniaDataset(Dataset):
    def __init__(self, root_dir, annotation_file, transform=None):
        self.root_dir = root_dir
        self.annotations = annotation_file
        self.transform = transform

    def __len__(self):
        return len(self.annotations)

    def __getitem__(self, index):
        img_id = (self.annotations.iloc[index, 0] + '.png')
        img = Image.open(os.path.join(self.root_dir, img_id)).convert("RGB")
        y_label = torch.tensor(float(self.annotations.iloc[index, 1]))

        if self.transform is not None:
            img = self.transform(img)

        return (img, y_label)

In [ ]:
batch_sz = 16

val_dataset_v3 = PneumoniaDataset(root, df_val, transform=transform_v3)
val_loader_v3 = torch.utils.data.DataLoader(dataset=val_dataset_v3, batch_size=batch_sz, shuffle=False)
test_dataset_v3 = PneumoniaDataset(root, df_test, transform=transform_v3)
test_loader_v3 = torch.utils.data.DataLoader(dataset=test_dataset_v3, batch_size=batch_sz, shuffle=True)

val_dataset_dense = PneumoniaDataset(root, df_val, transform=transform_dense)
val_loader_dense = torch.utils.data.DataLoader(dataset=val_dataset_dense, batch_size=batch_sz, shuffle=False)
test_dataset_dense = PneumoniaDataset(root, df_test, transform=transform_dense)
test_loader_dense = torch.utils.data.DataLoader(dataset=test_dataset_dense, batch_size=batch_sz, shuffle=True)

In [ ]:
from sklearn import metrics
from tqdm import tqdm
import torch.nn.functional as nnf

In [ ]:
import torch
torch.cuda.empty_cache()

InceptionV3

In [ ]:
model = torch.load('/content/drive/MyDrive/Saliency Map Research 2023/Models/thorax_resnet.pt')

In [ ]:
model = model.to(device)

In [ ]:
from torch.autograd import Variable
from tqdm import tqdm
model.eval()
prob1 = []
results = []
final_targets = []

with torch.no_grad():
  for num, data in enumerate(tqdm(test_loader_dense, desc='Testing')):
    imgs, label = data
    imgs, labels = imgs.to(device), label.to(device)
    test = Variable(imgs)
    output = model(test)
    ps = torch.exp(output)
    prob = nnf.softmax(output, dim=1)
    prob1 += prob.detach().cpu().numpy().tolist()
    top_p, top_class = prob.topk(1, dim=1)
    results += top_class.cpu().numpy().tolist()

    targets = labels.detach().cpu().numpy().tolist()
    final_targets.extend(targets)


In [ ]:
print(prob1)
prob2 = []
for x in prob1:
  prob2.append(x[1])

print(prob2)

In [ ]:
#print(final_targets)
#results = [item for sublist in results for item in sublist]
#print(results)
roc_auc = metrics.roc_auc_score(final_targets, results)
print(roc_auc)

In [ ]:
roc_auc = metrics.roc_auc_score(final_targets, prob2)
print(roc_auc)

In [ ]:
fpr, tpr, thresh = metrics.roc_curve(final_targets,  prob2, drop_intermediate=False)
#create ROC curve
plt.plot(fpr,tpr)
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()

DenseNet

In [ ]:
model = torch.load('/content/drive/MyDrive/Saliency Map Research 2023/Models/thorax_dense.pt')

In [ ]:
from torch.autograd import Variable
model.eval()
prob1 = []
results = []
final_targets = []
#names = []
print('===============================================start')
for num, data in enumerate(test_loader_v3):
     #print(num)
     #print("=====================================================")

     imgs, label = data
     imgs,labels = imgs.to(device), label.to(device)
     test = Variable(imgs)
     output = model(test)
     #print(output)
     ps = torch.exp(output)
     prob = nnf.softmax(output, dim=1)
     prob1 += prob.detach().cpu().numpy().tolist()
     #print(prob1[:1])
     top_p, top_class = prob.topk(1, dim = 1)
     results += top_class.cpu().numpy().tolist()

     targets = labels.detach().cpu().numpy().tolist()
     final_targets.extend(targets)

print('===============================================end')

In [ ]:
print(prob1)
prob2 = []
for x in prob1:
  prob2.append(x[1])

print(prob2)

In [ ]:
#print(final_targets)
#results = [item for sublist in results for item in sublist]
#print(results)
roc_auc = metrics.roc_auc_score(final_targets, results)
print(roc_auc)

In [ ]:
roc_auc = metrics.roc_auc_score(final_targets, prob2)
print(roc_auc)

In [ ]:
fpr, tpr, thresh = metrics.roc_curve(final_targets,  prob2, drop_intermediate=False)
#create ROC curve
plt.plot(fpr,tpr)
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()